In [1]:
import pandas as pd
import urllib.parse
def decode_url_and_remove_HTTPversion(encoded_url:str) -> str:
    return urllib.parse.unquote(str(encoded_url)).replace(" HTTP/1.1","").replace("GET /","/")
dataFrame:pd.DataFrame = pd.read_csv('./data/access.csv')
dataFrame = dataFrame[dataFrame['Referer'] != '-']
dataFrame['Request']=dataFrame['Request'].apply(decode_url_and_remove_HTTPversion)
dataFrame = dataFrame[dataFrame['Request'].str.contains('/browse/|/product/')]
dataFrame = dataFrame[~dataFrame['Request'].str.startswith('/image')]
dataFrame['Request'] = dataFrame['Request'].apply(lambda url: url.split('/')[-1])
dataFrame=dataFrame[dataFrame['Request']!='']
dataFrame['next_request'] = dataFrame.groupby('IP')['Request'].shift(-1)
dataFrame = dataFrame.dropna(subset=['next_request'])
dataFrame=dataFrame.head(1000)

In [2]:
import networkx as nx
# エッジのリストを作成し、遷移の数をカウント
transition_counts = dataFrame.groupby(['Request', 'next_request']).size().reset_index(name='count')
# グラフを作成し、エッジの属性として遷移の数を格納
G = nx.DiGraph()
for _, row in transition_counts.iterrows():
    if row['Request'] != row['next_request']:
        G.add_edge(row['Request'], row['next_request'], weight=row['count'])
def predict_next_products(G, current_product_url:str)->list[str]:
    # 現在の商品URLに対応するノードを見つける
    if current_product_url in G:
        # 次に見そうな商品のノードを取得する
        next_products = list(G.successors(current_product_url))
        return next_products
    else:
        # 現在の商品がグラフにない場合は空のリストを返す
        return []
test_request=dataFrame.head(3)
print("prediction experiment:")
for colmn,record in test_request.iterrows():
    print(record['Request']+" ---> ",end='')
    print(predict_next_products(G,record['Request']))
print("count ranking:")
sorted_transitions = transition_counts.sort_values('count', ascending=False)
print(sorted_transitions.head())  # 上位5件の遷移を表示

prediction experiment:
گوشت-کوب-برقی ---> ['لوازم-خانگی', 'گوشت-کوب-برقی-بوش-مدل-MSM881664']
یخچال-فریزر-فریزر-هایسنس-مدل-RS-20WC4SA ---> ['یخچال-فریزر-چهار-درب-سامسونگ-مدل-Romano-STS']
اجاق-گاز-مبله ---> ['اجاق-گاز-مبله-دوو-مدل-DGC5-111', 'اجاق-گاز-مبله-سینجر-مدل-S2060', 'اجاق-گاز-مبله-سینجر-مدل-S2086S', 'اجاق-گاز-مبله-سینجر-مدل-SG-ECO-5051', 'اجاق-گاز-مبله-سینجر-مدل-SG-P460STD', 'اجاق-گاز-مبله-هاردستون-مدل-میلتون-تک-فر', 'اجاق-گاز-مبله-هاردستون-مدل-میلتون-فول-آپشن-دو-فر', 'لوازم-بزرگ-آشپزخانه', 'لوازم-منزل']
count ranking:
                                        Request  \
480                                 لوازم-خانگی   
479                                 لوازم-خانگی   
320    رادیاتور-برقی-فن-دار-فلر-مدل-ORF-28130BK   
834  یخچال-فریزر-فریزر-پایین-بوش-مدل-KGN56LB304   
469                         لوازم-بزرگ-آشپزخانه   

                                   next_request  count  
480                                 لوازم-خانگی     11  
479                         لوازم-بزرگ-آشپزخانه 